In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install datasets
!pip install evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import torch

In [ ]:
ds = load_dataset("EdinburghNLP/xsum", trust_remote_code=True, streaming=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


0000.parquet:   0%|          | 0.00/304M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/17.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
sample = ds['train'][0]

In [ ]:
print(sample.keys())
print('')
print(sample['document'])
print('')
print(sample['summary'])

dict_keys(['document', 'summary', 'id'])

The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - an

In [ ]:
# summarizer = pipeline('summarization', model='facebook/bart-large-cnn') # This is good but not single sentence
# summarizer = pipeline('summarization', model="sshleifer/distilbart-cnn-12-6") # Not good
# summarizer = pipeline('summarization', model="sshleifer/distilbart-xsum-12-1") # Not good
summarizer = pipeline('summarization', model="google/pegasus-xsum") # Yup, this is pretty good
# summarizer = pipeline('summarization', model = 'facebook/bart-large-xsum') # Also decent

KeyboardInterrupt: 

In [ ]:
summary = summarizer(sample['document'])

In [ ]:
print(sample['summary'])
print('')
print(summary[0]['summary_text'])

del summarizer

In [ ]:
# Manual pipeline

# tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
# model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')
# tokenized_text = tokenizer(sample['document'], return_tensors='pt')
# output = model.generate(**tokenized_text, num_beams=4, max_length=150, early_stopping=True)
# decoded = tokenizer.decode(output[0], skip_special_tokens=True)
# print(decoded)

# del tokenizer, model

In [ ]:
sample2 = ds['train'][2]
print(sample2['document'])
print('')
print(sample2['summary'])

In [ ]:
bertscore = evaluate.load('bertscore')

In [ ]:
# Compare summarizer to reference
# predictions = [summary[0]['summary_text']]
# references = [sample['summary']]

# Compare full text to summary
references = [sample['document']]
predictions = [sample['summary']]

# Sanity check
# references = [sample['summary']]
# predictions = [sample['summary']]

# Sanity check
# references = [sample['document']]
# predictions = [sample2['document']]

# Sanity check
# references = [sample['summary']]
# predictions = [sample2['summary']]

# references = ['I hate you']
# predictions = ['I love you']

results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
results

In [ ]:
bleurt = evaluate.load("bleurt", module_type="metric")

In [ ]:
# Compare summarizer to reference
# predictions = [summary[0]['summary_text']]
# references = [sample['summary']]

# Compare full text to summary
references = [sample['document']]
predictions = [sample['summary']]

# Sanity check
references = [sample['summary']]
predictions = [sample['summary']]

# Sanity check
# references = [sample['document']]
# predictions = [sample2['document']]

# Sanity check
# references = [sample['summary']]
# predictions = [sample2['summary']]

# references = ['I hate you']
# predictions = ['I love you']

results = bleurt.compute(predictions=predictions, references=references)
results

In [ ]:
del bertscore, bleurt

In [ ]:
nli = pipeline("text-classification", model="FacebookAI/roberta-large-mnli")

In [ ]:
reference = sample['summary']
prediction = summary[0]['summary_text']

# Compare full text to summary
reference = sample['document']
prediction = sample['summary']

# Sanity check
reference = sample['summary']
prediction = sample['summary']

# Sanity check
reference = sample['document']
prediction = sample2['document']

# Sanity check
reference = sample['summary']
prediction = sample2['summary']

# reference = 'I hate you'
# prediction = 'I love you'

# reference = 'I like you'
# prediction = 'I love you'

# reference = 'I love you'
# prediction = 'I want to be with you'

print(f"{reference} </s> {prediction}")
results = nli(f"{reference} </s> {prediction}", truncation=True)
results

In [ ]:
del nli

In [ ]:
bleu = evaluate.load("bleu")

predictions = [summary[0]['summary_text']]
# predictions = [sample['summary']]
references = [sample['summary']]
results = bleu.compute(predictions=predictions, references=references)
results

In [ ]:
rouge = evaluate.load('rouge')

predictions = [summary[0]['summary_text']]
# predictions = [sample['summary']]
references = [sample['summary']]
results = rouge.compute(predictions=predictions, references=references)
results